In [ ]:
from_fp = 'mousebrainatlas-rawdata/CSHL_volumes/atlasV6/atlasV6_10.0um_scoreVolume/score_volumes/'
to_fp = '/media/alexn/BstemAtlasDataBackup/demo/CSHL_volumes/atlasV6/atlasV6_10.0um_scoreVolume/score_volumes/'
include_only = 'atlasV6_10.0um_scoreVolume_12N*'
execute_command('aws s3 cp --recursive \"s3://%(from_fp)s\" \"%(to_fp)s\" --exclude \"*\" --include \"%(include)s\"'\
                % dict(from_fp=from_fp, to_fp=to_fp, include=include_only))

In [ ]:
# Setup and Imports

# %reload_ext : Reloads an IPython extension by its module name.
%reload_ext autoreload
# %autoreload 2 : Reloads all modules (except those excluded by %aimport)  
#  every time before executing the Python code typed.
%autoreload 2

# %TEXT : code in this format is called a "magic function" 
#  https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline/43028034

import os
import sys
import time

import matplotlib.pyplot as plt
# Sets backend of matplotlib to the 'inline' backend
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *


stack = 'MD662'
print('testing metadata: ',metadata_cache['valid_filenames_all'][stack][0])

In [ ]:
# Download files from S3
# Not necessary if files are not on S3
! aws s3 cp --recursive --force-glacier-transfer \
    "s3://mousebrainatlas-rawdata/CSHL_data/MD662" "/shared/CSHL_data/MD662"

# Miscillaneous Code
#### Function to detect where you are at running the code

In [ ]:
# BAD FILE LOCATOR
# Automatically tells where in the code you are

# 1) raw -> raw_Ntb: extract_channel
# 2) raw_Ntb -> thumbnail_Ntb: rescale
# 3) thumbnail_Ntb -> thumbnail_NtbNormalized: normalize_intensity
# 4) Compute transforms using thumbnail_NtbNormalized: align + compose
# 5) Supply prep1_thumbnail_mask
# 6) prep1_thumbnail_mask -> thumbnail_mask: warp
# 7) raw_Ntb -> raw_NtbNormalizedAdaptiveInvertedGamma: brightness_correction
# 8) Compute prep5 (alignedWithMargin) cropping box based on prep1_thumbnail_mask
# 9) raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_raw_NtbNormalizedAdaptiveInvertedGamma: align + crop
# 10) thumbnail_NtbNormalized -> prep5_thumbnail_NtbNormalized: align + crop
# 11) prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_thumbnail_NtbNormalizedAdaptiveInvertedGamma: rescale
# 12) Specify prep2 (alignedBrainstemCrop) cropping box
# 13) prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGamma: crop
# 14) prep2_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg: compress_jpeg

# np.setdiff1d(list1,list2) # Get difference between two lists
# metadata_cache['valid_filenames_all'][stack]  # To get all valid filenames

from os import listdir
from PIL import Image

def where_am_i():
    '''
    where_am_i will iterate through each of the 14 steps of preprocessing, checking that ALL output files 
    have been generated sucessfully. For any stage where files are missing or corrupted, this function will 
    return a list of the filepaths & filenames of the images that are not in the output.
    where_am_i() returns [bad_fp_list, bad_file_list].
    '''
    
    # Throws an error if you don't set Max Pixels to around 2 Billion (yikes)
    Image.MAX_IMAGE_PIXELS = 2000000000
    # Iterates through every possible filename as defined in metadata_cache
    #                                           stack must be defined
    bad_fp_list = [] # Full filePaths
    bad_file_list = [] # FileNames
    
    #   Dic = {'step#': [prep_id, resol, version]}
    stepDic = {'1':[None,'raw','Ntb'], '2':[None,'thumbnail','Ntb'], '3':[None,'thumbnail','NtbNormalized'],\
               '4':[None,'???','???'], '5':[1,'thumbnail','mask'], '6':[None,'???','???'], \
               '7':[None,'raw','NtbNormalizedAdaptiveInvertedGamma'], '8':[5,'thumbnail','mask'], \
               '9':[5,'raw','NtbNormalizedAdaptiveInvertedGamma'], '10':[5,'thumbnail','NtbNormalized'], \
               '11':[5,'thumbnail','NtbNormalizedAdaptiveInvertedGamma']}

    
    # For checking each individual step of the pipeline as outlined above
    for step in range (2,12):
        stepStr = str(step)
        stepCompleted = True
        
        print 'Checking for completion of step '+stepStr
        t = time.time()
        
        if stepDic[stepStr][1]=='???':
                stepCompleted = False
                print 'Output files unknown. No idea.'
                print ''
                continue
        
        for fn in metadata_cache['valid_filenames_all'][stack]:
            
            # Record the filepath from every filename + info from the "step dictionary"
            fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=stepDic[stepStr][0], \
                                     resol=stepDic[stepStr][1], version=stepDic[stepStr][2], fn=fn)
            
            fp_to_test = fp
            try:
                img = Image.open(fp_to_test) # open the image file
                img.verify() # verify that it is, in fact an uncorruted image
            except (IOError, SyntaxError) as e:
               # print('Bad file:', fp_to_test) # Print out the names of corrupt files
                bad_fp_list.append(fp_to_test) # Add bad filepaths to bad_filepath_list
                bad_file_list.append(fn) # Add bad files to bad_file_list
                stepCompleted = False
        #metadata_cache['valid_filenames_all']['MD662']
        
        if stepCompleted:
            print 'Step '+stepStr+' completed!'
            print 'done in', time.time() - t, 'seconds' 
            print ''
        else:
            print '*****     Bad files found!     *****\n'
            print 'Step '+stepStr+' NOT complete. Rerun necessary parts of the code'
            print '`bad_fp_list` and `bad_file_list` created, each contain info on each incomplete files.'
            print 'done in', time.time() - t, 'seconds' 
            print ''
            if stepDic[stepStr][0] is None:
                print '`'+stack+'_'+stepDic[stepStr][1]+'_'+\
                        stepDic[stepStr][2]+'` was not generated properly'
            else:
                print '`'+stack+'_prep'+str(stepDic[stepStr][0])+'_'+stepDic[stepStr][1]+'_'+\
                        stepDic[stepStr][2]+'` was not generated properly'
            break
            #continue
    return bad_fp_list, bad_file_list

#print where_am_i.__doc__
[bad_fp_list, bad_file_list] = where_am_i()

print len(bad_file_list),' bad files'
print 'bad_fp_list and bad_file_list contain all of the problematic file names'

In [ ]:
# Specify raw file locations

# replaces running "initialize.py <input_spec_filepath>"

# The set of dirs where we should search for image files.
raw_data_dirs = \
{(None, 'raw'): '/media/alexn/BstemAtlasDataBackup/MD662/',
(None, 'down32'): '/media/alexn/BstemAtlasDataBackup/MD662/'}

# Specifies how to extract image name from file path.
# The first group returned by re.search is image_name.
input_image_filename_to_imagename_re_pattern_mapping = \
{(None, 'raw'): \
 '/media/alexn/BstemAtlasDataBackup/MD662/(.*)_lossless.jp2',
 (None, 'down32'): \
  '/media/alexn/BstemAtlasDataBackup/MD662/(.*).png', # Check raw data, may NOT be a .png file
}

# Check that every raw file has all 3 channels properly
image_names_all_data_dirs_flattened = set([])
image_names_all_data_dirs = {}
for vr, data_dir in raw_data_dirs.iteritems():
    if data_dir is None: continue
    image_names = set([])
    if vr in input_image_filename_to_imagename_re_pattern_mapping:
        for fn in os.listdir(data_dir):
            g = re.search(input_image_filename_to_imagename_re_pattern_mapping[vr], os.path.join(data_dir, fn))
            if g is not None:
                img_name = g.groups()[0]
                image_names.add(img_name)
                image_names_all_data_dirs_flattened.add(img_name)
    image_names_all_data_dirs[vr] = image_names
    
# Make sure the every image has all three channels.
for vr, img_names in image_names_all_data_dirs.iteritems():
    print vr, 'missing:'
    print image_names_all_data_dirs_flattened - img_names
    print 
    
# set([]) == Good

# Other instructions 
- You need to specify in metadata.py relevant filepaths. You need to set certain environment variables. User will need to specify stack name as well.
- If thumbnails (downsampled 32x) aren't provided run: resize.py [in_fp_map] [out_fp_map] 0.03125

# Specify input filepaths [From User Guide]
Create a JSON file that describes the image file paths as described in preprocessing.md: https://github.com/ActiveBrainAtlas/MouseBrainAtlas/blob/master/doc/User%20Manuals/user_guide_pages/Preprocessing.md

Then run `initialize.py <input_spec_filepath>`

thumbnail_NtbNormalized ->

                 -> <stack>_sorted_filenames.txt
 
                 -> <stack>_thumbnail_Ntb/
 
                 -> <stack>_raw_Ntb/

# STEP 1
### raw (.jp2) -> raw_Ntb (.tif)

In [ ]:
in_dir = '/media/alexn/BstemAtlasDataBackup/MD662/'
output_dir = create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=None, resol='raw'))

# INPUT TEST
in_list = [os.path.join(in_dir, img_name + '_lossless.jp2') 
                                       for img_name in list(image_names_all_data_dirs_flattened)]
# OUTPUT TEST
out_list = [DataManager.get_image_filepath_v2(stack=stack, prep_id=None, 
                                        resol='raw', version=None, fn=img_name) 
                                        for img_name in list(image_names_all_data_dirs_flattened)]

print('first input file: ',in_list[0])
print('first output file: ',out_list[0])

In [ ]:
# Multiple core processing on every individual file (takes about 4 minutes each)
# In total will take about 30 hours
# Creates new files at /CSHL_data_processed/MD662/MD662_raw/*_raw.tif
t = time.time()

run_distributed('export LD_LIBRARY_PATH=%(kdu_dir)s:$LD_LIBRARY_PATH; %(kdu_bin)s -i \"%%(in_fp)s\" -o \"%%(out_fp)s\"' % \
                {'kdu_bin': KDU_EXPAND_BIN, 'kdu_dir': os.path.dirname(KDU_EXPAND_BIN)},
                kwargs_list={'in_fp': [os.path.join(in_dir, img_name + '_lossless.jp2') 
                                       for img_name in list(image_names_all_data_dirs_flattened)], 
                             'out_fp': [DataManager.get_image_filepath_v2(stack=stack, prep_id=None, 
                                        resol='raw', version=None, fn=img_name) 
                                        for img_name in list(image_names_all_data_dirs_flattened)]},
                argument_type='single',
                jobs_per_node=1, # Use single process
                local_only=True, # Run local
                use_aws=False)   # Run local
print 'done in', time.time() - t, 'seconds' # 2252 seconds full stack

# STEP 2
### raw_Ntb -> thumbnail_Ntb
rescale

# & STEP 3
### thumbnail_Ntb -> thumbnail_NtbNormalized
normalize_intensity

In [ ]:
create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=None, resol='raw', version='Ntb'))

In [ ]:
# raw -> raw_Ntb
# Multiple core processing on every individual file (takes about 4 minutes each)
# In total will take about 20 hours
# Creates new files at CSHL_data_processed/MD662/MD662_raw_Ntb/*_raw_Ntb.tif
t = time.time()
run_distributed('convert \"%(in_fp)s\" -channel B -separate \"%(out_fp)s\"',
                kwargs_list={'in_fp': [DataManager.get_image_filepath_v2(stack=stack, prep_id=None, 
                                        resol='raw', version=None, fn=img_name) 
                                       for img_name in list(image_names_all_data_dirs_flattened)], 
                             'out_fp': [DataManager.get_image_filepath_v2(stack=stack, prep_id=None, 
                                        resol='raw', version='Ntb', fn=img_name) 
                                        for img_name in list(image_names_all_data_dirs_flattened)]},
                argument_type='single',
                jobs_per_node=1,
                local_only=True,
               use_aws=False)
print('finished generating raw_Ntb files')
print 'done in', time.time() - t, 'seconds' # 2252 seconds full stack

thumbnail_downscale_factor = 32
tb_resol = 'thumbnail'

# Takes about 1 minute per file, in total about 8-10 hours

# Will create new files at the filepaths /CSHL_data_processed/MD662/MD662_thumbnail_Ntb/*_thumbnail_Ntb.tif
#  and /CSHL_data_processed/MD662/MD662_thumbnail_NtbNormalized/*_thumbnail_NtbNormalized.tif
# Only 108 files in each directory though?
i = 0

for img_name in metadata_cache['valid_filenames_all'][stack]:
    i = i+1
    print '\n\n'+img_name+'\n'

    t = time.time()

    in_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, resol='raw', version='Ntb', \
                                              fn=img_name)
    out_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, resol=tb_resol, version='Ntb', \
                                              fn=img_name)
    create_parent_dir_if_not_exists(out_fp)
    
  #  if os.path.isfile(out_fp):
  #      print('SKIPPING NeurotraceB: '+out_fp)
  #      #continue
  #  else:
    
    try:
        img = imread(in_fp)
    except IndexError:
        print('Problematic file detected\n\n'+out_fp+'\n\n')
        
    print(out_fp)
    img_tb = img[::thumbnail_downscale_factor, ::thumbnail_downscale_factor]
    imsave(out_fp, img_tb)
    
    
    # Alternative: ImageMagick introduces an artificial noisy stripe in the output image.
#     cmd = 'convert %(in_fp)s -scale 3.125%% %(out_fp)s' % {'in_fp': in_fp, 'out_fp': out_fp}
#     execute_command(cmd)
        
    sys.stderr.write("Rescale: %.2f seconds.\n" % (time.time() - t)) # ~20s / image
    
    t = time.time()

    in_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, resol=tb_resol, version='Ntb', \
                                              fn=img_name)
    out_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, resol=tb_resol, version='NtbNormalized',\
                                               fn=img_name)
    create_parent_dir_if_not_exists(out_fp)
    
  #  if os.path.isfile(out_fp):
  #      print('SKIPPING Ntb Normalized: '+out_fp)
  #      continue
  #  else:
    print(out_fp)
    cmd = """convert "%(in_fp)s" -normalize -depth 8 "%(out_fp)s" """ % {'in_fp': in_fp, 'out_fp': out_fp}
    execute_command(cmd)
  #  try:
  #      img = imread(in_fp)
  #  except IndexError:
  #      print('Problematic file detected\n\n'+out_fp+'\n\n')
    
    
    sys.stderr.write("Intensity normalize: %.2f seconds.\n" % (time.time() - t))
print i

# STEP 4
### Compute transforms using thumbnail_NtbNormalized
align + compose

#### Generally how it works:
- Generate Elastix output from thumbnail_NtbNormalized
    - These are pairwise transformations for the files
        - Saved in /elastix_output/
    - Roughly 3% will fail and will need to be run through. Use GUI to fix
        - Saved in /custom_transform/
- Select an anchor file to unite all pairwise transformations
    - /gui/preprocess_tool_v3.py
        - Anchor.txt is created with name of anchor file
    - Transformation matrices generated for each file relative to anchor
        - Saved in transformTo_[anchorName].pkl
    - Generate /prep1_thumbnail_normalized/ images. May be padded with white
        - prep1_thumbnail_normalized files used as inputs to "Active Contour Algorithm" 
    

In [ ]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack, redownload=True) 

tb_fmt = 'tif'
version = 'NtbNormalized'
valid_filenames = metadata_cache['valid_filenames_all'][stack]
# Note that this could be the human-corrected version, in which case the transforms may not exist.
valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

script = os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py')
input_dir = DataManager.get_image_dir_v2(stack=stack, prep_id=None, version=version, resol='thumbnail')    
output_dir = create_if_not_exists(os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output'))
print('Input: ',input_dir)
print('Output: ',output_dir)

In [ ]:
t = time.time()
print 'Align...'

run_distributed("%(script)s %(stack)s \"%(input_dir)s\" \"%(output_dir)s\" \'%%(kwargs_str)s\' %(fmt)s -p \
%(param_fp)s -r" % \
                {'script': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir,
                'fmt': tb_fmt,
                 'param_fp': '/home/yuncong/Brain/preprocess/parameters/Parameters_Rigid_MutualInfo_noNumberOfSpatialSamples_4000Iters.txt'
                },
                kwargs_list=[{'prev_fn': valid_filenames[i-1] + '_thumbnail_' + version, 
                              'curr_fn': valid_filenames[i] + '_thumbnail_' + version,
                             'prev_sn': valid_filenames[i-1] ,
                             'curr_sn': valid_filenames[i] } 
                             for i in range(1, len(valid_filenames))],
                argument_type='list',
                jobs_per_node=8,
               local_only=True)

# wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 2252 seconds full stack

In [ ]:
# RUN GUI TO CHECK

# STEP 5
### Supply prep1_thumbnail_mask 
^ Need to clarify with Yuncong HOW to do this

#### Also compose alignments
-> [stackName]_transformsTo_[anchorFilename].pkl

thumbnail_NtbNormalized ->
[stackName]\_transformsTo\_[anchorFilename].pkl -> prep1_thumbnail_NtbNormalized

In [2]:
anchor_fn = DataManager.load_anchor_filename(stack=stack)
anchor_idx = valid_filenames.index(anchor_fn)
print 'anchor_idx =', anchor_idx

script = os.path.join(REPO_DIR, 'preprocess', 'compose_transform_thumbnail_v2.py')
input_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
output_fp = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                         dict(stack=stack, anchor_fn=anchor_fn))

print('Script: ',script,'\n')
print('Input: ',input_dir)
print('Output: ',output_dir)

NameError: name 'DataManager' is not defined

In [ ]:
# -> [stackName]_transformsTo_[anchorFilename].pkl
t = time.time()
print 'Composing transform...'

run_distributed("%(script)s %(stack)s \"%(input_dir)s\" \'%%(kwargs_str)s\' %(anchor_idx)d \"%(output_fp)s\"" % \
            {'stack': stack,
            'script': script,
            'input_dir': input_dir,
            'anchor_idx': anchor_idx,
            'output_fp': output_fp},
            kwargs_list=[{'filenames': metadata_cache['valid_filenames_all'][stack]}],
            argument_type='list',
               local_only=True)

# wait_qsub_complete()

print 'done in', time.time() - t, 'seconds' # 20 seconds

In [ ]:
t = time.time()
print 'Warping...'

transforms_to_anchor = DataManager.load_transforms(stack=stack, downsample_factor=32, \
                                                   use_inverse=False, anchor_fn=anchor_fn)
# useful for alternatively stained stacks where bg varies depending on stain on each section
if pad_bg_color == 'auto': 
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s \
    thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    },
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, \
                                                            prep_id=None, version=version, resol='thumbnail'),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn,\
                                                            prep_id=prep_id, version=version, resol='thumbnail'),
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in metadata_cache['valid_filenames_all'][stack]],
                    argument_type='single',
                   jobs_per_node=8,
                   local_only=True)
else:
    run_distributed('%(script)s %(stack)s \"%%(input_fp)s\" \"%%(output_fp)s\" %%(transform)s \
    thumbnail 0 0 2000 1500 %(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'input_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, \
                                                            prep_id=None, version=version, resol='thumbnail'),
                                  'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=fn, \
                                                            prep_id=prep_id, version=version, resol='thumbnail'),
                                 }
                                for fn in metadata_cache['valid_filenames_all'][stack]],
                    argument_type='single',
                   jobs_per_node=8,
                   local_only=True)

# wait_qsub_complete()
    
print 'done in', time.time() - t, 'seconds' # 300 seconds

# STEP 6
### prep1_thumbnail_mask -> thumbnail_mask
warp

[
### Next step is to run the "Active Conour Algorithm", the mask generation step.
prep1_thumbnail_normalized -> prep1_thumbnail_mask
### For now, I am skipping this step. Y suggested I download masks from S3 directly and skip to the alignment process.
Skipping Mask Generation. Downloading masks.
]

In [ ]:
download_from_s3(os.path.join('CSHL_data_processed', stack, stack + '_prep1_thumbnail_mask'), 
                 is_dir=True, local_root=DATA_ROOTDIR, redownload=True)

# STEP 7
### raw_Ntb -> raw_NtbNormalizedAdaptiveInvertedGamma
brightness_correction

(Requires *_raw_normalizedFloatMap.bp' for each file)
generate raw_mask -> generate STACK_intensity_normalization_results/normalizedFloatMap/ files

In [ ]:
from skimage.exposure import rescale_intensity, adjust_gamma

# Generates the raw_mask
for section in metadata_cache['valid_sections_all'][stack]:
    
    print "Section", section
    
    t = time.time()
    
    img = DataManager.load_image_v2(stack=stack, prep_id=None, section=section, version='Ntb', resol='raw')

    sys.stderr.write('Load image: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=None, section=section)
#     raw_mask = rescale_by_resampling(tb_mask, new_shape=(img.shape[1], img.shape[0]))
    raw_mask = resize(tb_mask, img.shape) > .5
    
    save_data(raw_mask, 
          DataManager.get_image_filepath_v2(stack=stack, prep_id=None, section=section, version='mask', resol='raw', ext='bp'), 
          upload_s3=False)
    
    sys.stderr.write('Rescale mask: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    
    mean_std_all_regions = []
    cx_cy_all_regions = []
    region_size = 5000
    region_spacing = 3000
    for cx in range(0, img.shape[1], region_spacing):
        for cy in range(0, img.shape[0], region_spacing):
            region = img[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                         max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            region_mask = raw_mask[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                                   max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            if np.count_nonzero(region_mask) == 0:
                continue
            mean_std_all_regions.append((region[region_mask].mean(), region[region_mask].std()))
            cx_cy_all_regions.append((cx, cy))
            
    sys.stderr.write('Compute mean/std for sample regions: %.2f seconds.\n' % (time.time() - t))
    
    t = time.time()
    mean_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,0], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4, 
                                 interpolation_order=2)

    sys.stderr.write('Interpolate mean map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    mean_map = rescale_by_resampling(mean_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up mean map: %.2f seconds.\n' % (time.time() - t)) #30s

    t = time.time()
    std_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,1], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4,
                               interpolation_order=2)
    sys.stderr.write('Interpolate std map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    std_map = rescale_by_resampling(std_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up std map: %.2f seconds.\n' % (time.time() - t)) #30s
    
    # Save mean/std results.
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='region_centers', stack=stack, section=section)
    create_parent_dir_if_not_exists(fp)    
    np.savetxt(fp, cx_cy_all_regions)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='mean_std_all_regions', stack=stack, section=section)
    create_parent_dir_if_not_exists(fp)
    np.savetxt(fp, mean_std_all_regions)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='mean_map', stack=stack, section=section)
    create_parent_dir_if_not_exists(fp)
    bp.pack_ndarray_file(mean_map.astype(np.float16), fp)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='std_map', stack=stack, section=section)
    create_parent_dir_if_not_exists(fp)
    bp.pack_ndarray_file(std_map.astype(np.float16), fp)

    # Export normalized image.
    
    t = time.time()
    raw_mask = raw_mask & (std_map > 0)
    img_normalized = np.zeros(img.shape, np.float32)
    img_normalized[raw_mask] = (img[raw_mask] - mean_map[raw_mask]) / std_map[raw_mask]
    sys.stderr.write('Normalize: %.2f seconds.\n' % (time.time() - t)) #30s

    t = time.time()
    # FIX THIS! THIS only save uint16, not float16. Need to save as bp instead.
#     img_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, \
#version='NtbNormalizedFloat', resol='down8', section=section, )
#     create_parent_dir_if_not_exists(img_fp)
#     imsave(img_fp, img_normalized[::8, ::8].astype(np.float16))
    save_data(img_normalized.astype(np.float16), 
              DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, section=section),
             upload_s3=False)
    sys.stderr.write('Save float version: %.2f seconds.\n' % (time.time() - t)) #30s
    
    
    # Export histogram.
    
    plt.hist(img_normalized[raw_mask].flatten(), bins=100, log=True);
    fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram_png', stack=stack, section=section)
    create_parent_dir_if_not_exists(fp)
    plt.savefig(fp)
    plt.close();
    
#     hist_fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram', stack=stack, section=section)
#     create_parent_dir_if_not_exists(hist_fp)
    
#     hist, bin_edges = np.histogram(img_normalized[valid_mask].flatten(), bins=np.arange(0,201,5));

#     plt.bar(bin_edges[:-1], np.log(hist));
#     plt.xticks(np.arange(0, 200, 20), np.arange(0, 200, 20));
#     plt.xlabel('Normalized pixel value (float)');
#     plt.title(metadata_cache['sections_to_filenames'][stack][section])

#     plt.savefig(hist_fp)
#     plt.close();

In [ ]:
# 5-10 seconds per section
for section in metadata_cache['valid_sections_all'][stack]:
        
    print section
    
    raw_mask = load_data(DataManager.get_image_filepath_v2(stack=stack, prep_id=None, \
                                     section=section, version='mask', resol='raw', ext='bp'),download_s3=False)
    
    img_normalized = load_data(
              DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', \
                                     stack=stack, section=section), download_s3=False)
        
    q = img_normalized[raw_mask]
    
    save_data(np.percentile(q, range(101)), 
              DataManager.get_intensity_normalization_result_filepath(what='float_percentiles', \
                                                    stack=stack, section=section), upload_s3=False)

In [ ]:
gamma_map = img_as_ubyte(adjust_gamma(np.arange(0, 256, 1) / 255., 8.))
# 10-30 seconds per section
for section in metadata_cache['valid_sections_all'][stack]:

    print section
    
    img_normalized = load_data(
              DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', \
                                                        stack=stack, section=section), download_s3=False)
        
    t = time.time()
    img_normalized_uint8 = rescale_intensity_v2(img_normalized, -2., 50.)
    sys.stderr.write('Rescale to uint8: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    raw_mask = load_data(DataManager.get_image_filepath_v2(stack=stack, prep_id=None, section=section, \
                                                    version='mask', resol='raw', ext='bp'), download_s3=False)
    img_normalized_uint8[~raw_mask] = 0
    sys.stderr.write('Load mask: %.2f seconds.\n' % (time.time() - t))
    
#     t = time.time()
#     save_data(img_normalized_uint8, DataManager.get_image_filepath_v2(stack=stack, prep_id=None, \
#section=section, version='NtbNormalizedAdaptive', resol='raw'),
#              upload_s3=False)
#     sys.stderr.write('Save uint8 version: %.2f seconds.\n' % (time.time() - t))
    
#     img_normalized_uint8 = \
#     DataManager.load_image_v2(stack=stack, prep_id=None, section=section, version='NtbNormalizedAdaptive',\
#resol='raw')
    img = 255 - img_normalized_uint8
    save_data(gamma_map[img], 
              DataManager.get_image_filepath_v2(stack=stack, prep_id=None, section=section, \
                                    version='NtbNormalizedAdaptiveInvertedGamma', resol='raw'), upload_s3=False)
# outputs the *_raw_NtbNormalizedAdaptiveInvertedGamma files

# STEP 8
### prep1 -> prep5
Compute prep5 (alignedWithMargin) cropping box based on prep1_thumbnail_mask

# STEP 9
### raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_raw_NtbNormalizedAdaptiveInvertedGamma
align + crop

In [ ]:
# roughly 1 second per slice
# Generates *_prep5_raw_NtbNormalizedAdaptiveInvertedGamma
for stack in ['MD662']:

    alignedWithMargin_xmin, alignedWithMargin_xmax,\
    alignedWithMargin_ymin, alignedWithMargin_ymax = DataManager.load_cropbox_v2(stack=stack, anchor_fn=None, 
                                                            prep_id='alignedWithMargin',
                                                           return_dict=False, only_2d=True)
            
    for section in metadata_cache['valid_sections_all'][stack]:
        
        for version in ['NtbNormalized']:
#         for version in [None, 'mask']:

            in_fp = DataManager.get_image_filepath_v2(\
                                stack=stack, prep_id=1, section=section, version=version, resol='thumbnail')

            out_fp = DataManager.get_image_filepath_v2(\
                                stack=stack, prep_id=5, section=section, version=version, resol='thumbnail')

            create_parent_dir_if_not_exists(out_fp)

            t = time.time()

            im_prep1 = imread(in_fp)
            im_prep5 = im_prep1[alignedWithMargin_ymin:alignedWithMargin_ymax+1, 
                                alignedWithMargin_xmin:alignedWithMargin_xmax+1]        
            save_data(im_prep5, out_fp)
            
            sys.stderr.write('Generate prep5: %.2f seconds.\n' % (time.time() - t))

# STEP 10
### thumbnail_NtbNormalized -> prep5_thumbnail_NtbNormalized
align + crop

# STEP 11
### prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_thumbnail_NtbNormalizedAdaptiveInvertedGamma
rescale

# STEP 12
### Specify prep2 (alignedBrainstemCrop) cropping box


# STEP 13
### prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGamma
crop

# STEP 14
### prep2_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg
compress_jpeg